# Testing the Model Deployment with your WebCam 📹

After deploying the model using RHODS Model Serving, we'd like to test the model deployment by sending images to the model server for real-time inference.

In this notebook we'll review how to consume the model through the RHODS Model Server.

For testing the model deployment, our test script needs to know the address of the model server. Let's insert the **inference endpoint** that the RHODS Dashboard provides for the deployed model.

In [1]:
prediction_url = 'https://model-team-1-ai.apps.cluster-djsg4.djsg4.sandbox703.opentlc.com/v2/models/model/infer'
token = 'eyJhbGciOiJSUzI1NiIsImtpZCI6InZ2NFZUTVRJS2wwYnA5RXVhcHY5UWJBSzFmOUpaY2pxODhPejhhck5WWkUifQ.eyJpc3MiOiJrdWJlcm5ldGVzL3NlcnZpY2VhY2NvdW50Iiwia3ViZXJuZXRlcy5pby9zZXJ2aWNlYWNjb3VudC9uYW1lc3BhY2UiOiJ0ZWFtLTEtYWkiLCJrdWJlcm5ldGVzLmlvL3NlcnZpY2VhY2NvdW50L3NlY3JldC5uYW1lIjoiZGVmYXVsdC1uYW1lLW1vZGVsLXNlcnZlci1zYSIsImt1YmVybmV0ZXMuaW8vc2VydmljZWFjY291bnQvc2VydmljZS1hY2NvdW50Lm5hbWUiOiJtb2RlbC1zZXJ2ZXItc2EiLCJrdWJlcm5ldGVzLmlvL3NlcnZpY2VhY2NvdW50L3NlcnZpY2UtYWNjb3VudC51aWQiOiJmYWI5NDVmOC1hMGI0LTRkYjEtYTFkMS0zMjZhZGFiYWRmOWQiLCJzdWIiOiJzeXN0ZW06c2VydmljZWFjY291bnQ6dGVhbS0xLWFpOm1vZGVsLXNlcnZlci1zYSJ9.SXovPphySt-8pBsmNJwCn4jsPVlx7yww0gSvqk0Ym6KYus2fdBwwbbrdLPDwKVNC21sDIUZegyB-5I2ms0acooEZCdHU6y3jRQAp_ml8nridVIZzNZqWRTCx3toS3z7gV8x8A_daRDUVCVariFtVizhd6Loz78iUDY_jUWWbJT6MnKMM65LBXvsQqRjFWiIZTSe1x21K4FzdNjKvVKlcTfCSYdfp3Iuc7r5VKcat0JxBO7Vp8yFDffuQHihBCd_UsAMpvN7yRZEj2GGfGbV9yh-R-Y5aZl_1Tlw-34xtXClp_odHZ4Iu11DBRrL4__auzvl4Sf-zB4FLt3eRG22A3YL6Z2NwrDZ4z7q9BrR17z7o99CG-ZWMM2xy7xiTRky69Az0efhM84ZiX9TXyJoraf9hu_znuV9NUpBAoAdiC7ASvg6fHg20kddOdAfOu6EfZPqrbLgY2pOPnotGWoYBb89hoL-0q15QnBfJ-daSu5S_dSzbTTAzmrGzTll1gt3V5anRBmls3MGMyAupy1Oz3AhdppIgJtrm-JmuXsIet_7GJxrOxAqUjtd3rto9B1up_AFHWuxScTU4CxuKJd90d2ssbYLZrJv_WaTtPbF0hDIuhZ2yN7XJwhB-ov6ZPRQFZUUy6fpvgsuEWp-zksgmP29IBE8lcpUp1ICrV2BlgbY'

We'll start by importing the preprocessing and rendering functions that we have worked with in the previous notebook.

In [2]:
import numpy as np
from jupyter_ui_poll import ui_events
import ipywidgets as widgets
from requests import post
import torch
import time

from preprocessing import preprocess_image_file,_transform

from object_detection import postprocess
from object_rendering import draw_boxes
import yaml
import PIL.Image
import io

from ipywebrtc import CameraStream, ImageRecorder
camera = CameraStream(constraints=
                      {'facing_mode': 'user',
                       'audio': False,
                       'video': { 'width': 640, 'height': 480 }
                       })
# camera

## Lets take a Picture

with the little 📷 icon.

In [3]:
image_recorder = ImageRecorder(stream=camera)
image_recorder.recording = True
image_recorder.autosave = False
image_recorder.download()
image_recorder.image.height
image_recorder

ImageRecorder(image=Image(value=b''), recording=True, stream=CameraStream(constraints={'facing_mode': 'user', …

## Click "Continue" button to continue with your picture

In [4]:
button = widgets.Button(description = "Continue")
output = widgets.Output()
display(button, output)

clicked = False
def on_button_clicked(b):
    global clicked
    clicked = True
    with output:
        image_recorder.save('webcam')

button.on_click(on_button_clicked)

with ui_events() as poll:
    while not clicked:
        poll(10) # poll queued UI events including button
        time.sleep(1) # wait for 1 second before checking again
        
print("Let's proceed")

In [ ]:
image_path = 'webcam.png'
transformed_image, scaling, padding = preprocess_image_file(image_path)

We'll now need to package the preprocessed image into a format that the model server can consume. RHODS Model Serving implements a generic prediction interface that allows to query the typical model formats through the HTTP POST method using a JSON request body.

In [ ]:
def serialize(image):
    payload = {
        'inputs': [
            {
                'name': 'images',
                'shape': [1, 3, 640, 640],
                'datatype': 'FP32',
                'data': image.flatten().tolist(),
            }
        ]
    }
    return payload

In [ ]:
payload = serialize(transformed_image)

Let's now send the serialized cat image to the model server. The inference results will also be returned in a generic JSON structure, which we can unpack straightaway. We'll also apply the post-processing function we defined in the previous notebook to extract the familiar object properties.

In [ ]:
def get_model_response(payload, prediction_url, classes_count, token=''):
    headers = {'Authorization': f'Bearer {token}'} if token else {}
    raw_response = post(prediction_url, json=payload, headers=headers)
    try:
        response = raw_response.json()
    except:
        print(f'Failed to deserialize service response.\n'
              f'Status code: {raw_response.status_code}\n'
              f'Response body: {raw_response.text}')
    try:
        model_output = response['outputs']
    except:
        print(f'Failed to extract model output from service response.\n'
              f'Service response: {response}')
    unpacked_output = _unpack(model_output, classes_count)
    return unpacked_output


def _unpack(model_output, classes_count):
    arr = np.array(model_output[0]['data'])
    # Get the response data as a NumPy Array

    output = torch.tensor(arr)  # Create a tensor from array
    prediction_columns_number = 5 + classes_count
    # Model returns model returns [xywh, conf, class0, class1, ...]

    output = output.reshape(
        1,
        int(int(output.shape[0])/prediction_columns_number),
        prediction_columns_number
    )  # Reshape the flat array prediction

    return output

def _read_class_labels(configuration_file_path):
    with open(configuration_file_path, 'r') as config_file:
        config = yaml.load(config_file.read(), Loader=yaml.SafeLoader)

    class_labels = config['names']
    return class_labels

In [ ]:
class_labels = _read_class_labels('model-training/configuration.yaml')
raw_objects = get_model_response(payload, prediction_url, len(class_labels), token=token)
objects = postprocess(raw_objects, conf_thres=0.3)
objects

Let's now visualize the result as in the previous notebook.

In [ ]:
draw_boxes(image_path, objects, scaling, padding, class_labels)

We were able to reproduce the object detection example from the previous notebook, so we can consume the deployed model as expected.

You can now head over to deploying the object detection application to consume this model in a real-time fashion.

After that, we'll explore offline scoring in the next notebook.